In [33]:
import pandas as pd
import numpy as np
import os, sys
from collections import defaultdict, Counter

In [26]:
df_session = pd.read_csv("data/session/session_reduced_2016-04-01.csv", sep = ";")
df_purchase = pd.read_csv("data/purchase/purchase_2016-04-01.csv", sep = ";")

In [27]:
df_session.shape

(273542, 3)

In [28]:
df_purchase.shape

(27119, 3)

In [30]:
session_dict = {}
for my_id,search_id, prop60 in df_session.values:
    key = (my_id, search_id)
    session_dict[key] = prop60.split(":")

In [36]:
tot = []
for my_id, search_id, product_id_buy in df_purchase.values:
    key = (my_id, search_id)
    if key in session_dict:
        top6 = session_dict[key]
        if product_id_buy in top6:
            pos = top6.index(product_id_buy)
            tot.append(pos)
        else:
            tot.append(100)

In [37]:
cnt = Counter(tot)

In [38]:
cnt.most_common()

[(100, 3813), (0, 2938), (1, 1723), (2, 1218), (3, 895), (4, 636), (5, 406)]